# Populando o Pinecone com Base de Conhecimento
Indexação de informações turísticas de Rio de Janeiro e Paris para uso no RAG.

In [ ]:
import numpy as np
from pinecone import Pinecone
pinecone_client = Pinecone()
indice = pinecone_client.Index('turismo')

In [ ]:
# Carregar base de conhecimento e dividir em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np
import os

# Carregar base de conhecimento
with open('base_conhecimento_rio_paris.txt', 'r', encoding='utf-8') as f:
    texto = f.read()

# Separar em chunks (ajuste chunk_size conforme necessário)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.create_documents([texto])

# Gerar embeddings para cada chunk
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

ids = [f'chunk_{i}' for i in range(len(chunks))]
vetores = [embeddings_model.embed_documents([chunk.page_content])[0] for chunk in chunks]

# Popular o Pinecone com os embeddings reais
indice.upsert(vectors=list(zip(ids, vetores)))

print(f"{len(chunks)} chunks indexados no Pinecone!")